# Leaf Classfication 
A For CZ4041 Machine Learning Assignment from PT3 in AY2018/2019 Semester 2.
The group members are:
- LIU Yiqing
- LUO Bingyi
- TENG He Xu
- WANG Jia
- YI Zhiyue
- ZHAO Ziru

## Import necessary libraries and Define Constants

In [2]:
import os
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder, StandardScaler

LABEL_PATH = 'data/'
TRAIN_FILE_NAME = 'train.csv'
TEST_FILE_NAME = 'test.csv'


Using TensorFlow backend.


## Load From CSV

Load features and labels from the train csv file

In [3]:
train_data_frame = pd.read_csv(LABEL_PATH + TRAIN_FILE_NAME)

train_data_frame = train_data_frame.drop(['id'], axis=1)

train_y = train_data_frame.pop('species')
classes = np.unique(train_y)

train_y = to_categorical(LabelEncoder().fit(train_y).transform(train_y))

#train_x = StandardScaler().fit(train_data_frame).transform(train_data_frame)
train_x = train_data_frame.get_values()

In [4]:
no_of_classes = len(np.unique(train_y, axis=0))

In [5]:
train_x

array([[ 0.007812,  0.023438,  0.023438, ...,  0.004883,  0.      ,
         0.025391],
       [ 0.005859,  0.      ,  0.03125 , ...,  0.000977,  0.039062,
         0.022461],
       [ 0.005859,  0.009766,  0.019531, ...,  0.      ,  0.020508,
         0.00293 ],
       ..., 
       [ 0.001953,  0.003906,  0.      , ...,  0.027344,  0.      ,
         0.001953],
       [ 0.      ,  0.      ,  0.046875, ...,  0.      ,  0.001953,
         0.00293 ],
       [ 0.023438,  0.019531,  0.03125 , ...,  0.023438,  0.025391,
         0.022461]])

## Build model

In [34]:
model = Sequential()

model.add(Dense(250, activation='relu', input_dim = train_x.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(no_of_classes, activation=tf.nn.softmax))

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 250)               48250     
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 150)               37650     
_________________________________________________________________
dropout_2 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 99)                14949     
Total params: 100,849
Trainable params: 100,849
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model.fit(train_x, train_y, epochs = 50)

Epoch 1/50
990/990 [==============================] - 0s 117us/step - loss: 1.8371 - acc: 0.4556
Epoch 2/50
990/990 [==============================] - 0s 100us/step - loss: 1.7286 - acc: 0.4949
Epoch 3/50
990/990 [==============================] - 0s 93us/step - loss: 1.5429 - acc: 0.5545
Epoch 4/50
990/990 [==============================] - 0s 93us/step - loss: 1.4244 - acc: 0.5879
Epoch 5/50
990/990 [==============================] - 0s 97us/step - loss: 1.3313 - acc: 0.6152
Epoch 6/50
990/990 [==============================] - 0s 109us/step - loss: 1.2677 - acc: 0.6323
Epoch 7/50
990/990 [==============================] - 0s 97us/step - loss: 1.1717 - acc: 0.6535
Epoch 8/50
990/990 [==============================] - 0s 91us/step - loss: 1.0982 - acc: 0.6747
Epoch 9/50
990/990 [==============================] - 0s 92us/step - loss: 1.0308 - acc: 0.6919
Epoch 10/50
990/990 [==============================] - 0s 93us/step - loss: 0.9800 - acc: 0.7091
Epoch 11/50
990/990 [===============

## Test

In [39]:
test_data_frame = pd.read_csv(LABEL_PATH + TEST_FILE_NAME)

index = test_data_frame.pop('id')

#test_x = StandardScaler().fit(test_data_frame).transform(test_data_frame)
test_x = test_data_frame.get_values()

In [40]:
test_y = model.predict_classes(test_x)

In [41]:
test_y

array([51, 50,  1, 19, 14,  3,  3, 28, 84,  8, 43, 74, 75, 10, 52, 60, 45,
       73, 13, 71, 61, 68, 57, 77,  1, 70, 28, 15, 35, 70, 53, 74, 47, 50,
        4, 36, 14, 55, 36, 93,  8, 32,  8,  9, 71, 70, 38, 23, 94, 18, 17,
        5, 55, 94, 14, 86, 62, 33, 51, 98, 88, 56, 21, 59, 65, 11, 48,  5,
       13,  4, 54, 57, 29,  7, 31, 98, 92, 84, 25, 10, 61, 43, 85, 24,  1,
        2, 23, 83, 40, 22, 48, 90, 25, 21, 37, 56, 41, 95,  7, 49, 98, 77,
        3, 12, 31, 84, 53, 96, 64, 72, 93, 93, 67, 30,  8, 88, 60, 87,  6,
       57, 34, 34, 60, 17, 75, 27, 51, 88, 39, 23, 23,  2, 41, 61, 24, 97,
       29, 28, 68, 81, 42, 51, 86, 62, 60, 52, 95, 81, 42, 96, 95, 20, 59,
       35, 86,  1, 26, 38, 43, 75, 20, 60, 46, 79, 22, 79, 69, 87, 65, 97,
       75, 21, 29, 21, 11, 10, 58, 94, 27, 22, 15, 45, 89, 54, 43,  5, 23,
       94, 40, 49, 89, 72, 36, 89, 81, 95, 18, 91, 29, 64, 80,  6, 78, 45,
       28,  9, 78, 90, 44, 89, 92, 13,  2, 59,  0, 96, 70, 32, 29, 78, 91,
       55, 44, 38,  5, 60

In [42]:
data_grid = np.zeros((len(test_y), len(classes)))

for n in range(len(test_y)):
    data_grid[n][test_y[n]] = 1
    


In [43]:
prediction = pd.DataFrame(data_grid, index = index, columns = classes)

In [44]:
np.shape(test_y)

(594,)

In [45]:
with open('submission.csv','w') as file:
    file.write(prediction.to_csv())
